In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os, cv2
from osgeo import gdal, osr, ogr
from tqdm import tqdm_notebook

In [2]:
data_dir = "/Users/mac/Desktop/昌邑大棚/"
tif_path = os.path.join(data_dir, 'L17.tif')
rgb_image = []
raster = gdal.Open(tif_path)
for band_number in [1,2,3]:
    band = raster.GetRasterBand(band_number)
    rgb_image.append(band.ReadAsArray())
rgb_image = np.stack(rgb_image, axis=-1)

In [3]:
square = 160
n_row = rgb_image.shape[0]/square
n_col = rgb_image.shape[1]/square

In [4]:
def save_pic(index, image=rgb_image):
    from PIL import Image
    slice_index, pos_index = index
    Image.fromarray(image[slice_index]).save("/Users/mac/Desktop/昌邑大棚/pieces/image_{:03}_{:03}.png".format(*pos_index))

In [5]:
pic_index = []
for i in range(n_row):
    for j in range(n_col):
        pic_index.append(((slice(i*square, i*square+square), slice(j*square, j*square+square)), (i, j)))

In [6]:
for p in tqdm_notebook(pic_index, desc=u"剪裁图片"):
    save_pic(p)

In [10]:
from PIL import Image
Image.fromarray(rgb_image).save("changyi.jpg")

In [11]:
a = Image.open("changyi.jpg")

In [13]:
b = np.asarray(a)

In [14]:
b.shape

(23040, 28928, 3)